In [ ]:
# reading data
## Example with GridSearch CV
#https://stackoverflow.com/questions/38555650/try-multiple-estimator-in-one-grid-search/53292354
import csv
import re
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import f1_score, make_scorer, classification_report
import pandas as pd
from sklearn.metrics import classification_report

In [ ]:
path = 'hatespeech_text_label_vote_RESTRICTED_100K.csv'
!head hatespeech_text_label_vote_RESTRICTED_100K.csv
#tweet, label, number of voters

In [ ]:
tweets = []
labels = []
with open(path) as fi:
    data = csv.reader(fi, delimiter='\t')
    for row in data:
        tweets.append(row[0])
        labels.append(row[1]) 

In [ ]:
## sanity checks
len(tweets) == len(labels)
print(len(tweets))

In [ ]:
# Classes are not balanced... this has implications for the training procedure.
Counter(labels)

In [ ]:
plt.bar(Counter(labels).keys(), Counter(labels).values())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.2, random_state=42) 


In [ ]:
y_train[0]

In [ ]:
X_train[0]


In [ ]:
# one example solution:
pipeline = Pipeline(steps = [('vectorizer', TfidfVectorizer()), ('classifier', LogisticRegression(solver='liblinear'))])

grid = {
    'vectorizer__ngram_range' : [(1,1), (1,2)],
    'vectorizer__max_df': [0.5, 1.0],
    'vectorizer__min_df': [0, 5],
    'classifier__C': [0.01, 1, 100]
}

f1 = make_scorer(f1_score, average='macro') # macro --> emphasizes the prediction of the smaller classes (in this case: hateful/ spam)

search = GridSearchCV(estimator=pipeline,
                      param_grid=grid,
                      #scoring='accuracy', 
                      scoring=f1,     # note that this is *not* the string 'f1', but the scorer object we constructed above
                      cv=3,
                      n_jobs=-1,  # use all cpus
                      verbose=10)
                      
search.fit(X_train, y_train)
print(f'Using these hyperparameters {search.best_params_}, we get the best performance.')


In [ ]:
print(classification_report(y_test, search.predict(X_test)))

#### Other options to try: 
- Different preprocessing steps (e.g., stopword removal, different tokenizers, stemming, lemmatization, ...)
- Different vectorizers (e.g. CountVectorizer())
- Different classifiers (e.g., MultinomialNB(), SVC())
- Class weights
- ...